In [1]:
index_file = "/home/szymon/lingwy/nkjp/nkjp_index.txt"
nkjp_path = "/home/szymon/lingwy/nkjp/pełny/"
vecs_path = "/home/szymon/lingwy/nkjp/wektory/nkjp+wiki-lemmas-all-100-skipg-ns.txt/"
skladnica_sections_index = "/home/szymon/lingwy/nkjp/skladnica_znacz/sections.txt"
skladnica_path = "/home/szymon/lingwy/nkjp/skladnica_znacz/"
pl_wordnet = "/home/szymon/lingwy/wielozn/wzięte/plwordnet_2_0_4/plwordnet_2_0_4_release/plwordnet_2_0.xml"
vecs_dim = 100
window_size = 4 # how many words to condider on both sides of the target
batch_size = window_size * 2
corp_runs = 2
learning_rate = 0.3
reg_rate = 0.005
points_per_neg_sample = 1

## Get Składnica & NKJP sentences

In [2]:
skl_ids = []
nkjp_orig_ids = []
nkjp_squeezed_ids = [] # Składnica uses "squeezed" version, w/o hyphens (-)

# This will be used later. NKJP id -> [ sent_ids ]
skl_nkjp_sent_ids = {}
# auxiliary, skl_id -> NKJP_id
nkjp_skl_ids = {}

with open(skladnica_sections_index) as index:
    for sec_name in index:
        skl_ids.append(sec_name.strip())
with open(index_file) as index:
    for fragm_id in index:
        fragm_id = fragm_id.strip()
        nkjp_orig_ids.append(fragm_id)
        nkjp_squeezed_ids.append(fragm_id.replace('-', ''))
        
bonus_skladnica_mappings = {
                            'NKJP_1M_001000-DziennikPolski1980': 'DP1980',
                            'NKJP_1M_010200-ZycieWarszawyPLUSZycie': 'ZycieWarszawy_Zycie',
                            'NKJP_1M_012750-TrybunaPLUSTrybunaLudu': 'TrybunaLudu_Trybuna',
                            'NKJP_1M_1998': 'DP1998',
                            'NKJP_1M_1999': 'DP1999',
                            'NKJP_1M_2000': 'DP2000',
                            'NKJP_1M_2001': 'DP2001',
                            'NKJP_1M_2002': 'DP2002',
                            'NKJP_1M_2003': 'DP2003',
                            'NKJP_1M_%2E': '712-3-900001'
                           }

In [3]:
# This should print Składnica ids for which we don't know their NKJP equivalents.
# (hopefully, none)
for id in skl_ids:
    if id[len('NKJP_1M_'):] in nkjp_squeezed_ids:
        sqzd_nkjp_id = id[len('NKJP_1M_'):]
        orig_nkjp_id = nkjp_orig_ids[nkjp_squeezed_ids.index(sqzd_nkjp_id)]
        skl_nkjp_sent_ids[orig_nkjp_id] = []
        nkjp_skl_ids[id] = orig_nkjp_id
    elif id[len('NKJP_1M_4scal-'):] in nkjp_squeezed_ids:
        sqzd_nkjp_id = id[len('NKJP_1M_4scal-'):]
        orig_nkjp_id = nkjp_orig_ids[nkjp_squeezed_ids.index(sqzd_nkjp_id)]
        skl_nkjp_sent_ids[orig_nkjp_id] = []
        nkjp_skl_ids[id] = orig_nkjp_id
    elif id[len('NKJP_1M_######-'):] in nkjp_squeezed_ids:
        sqzd_nkjp_id = id[len('NKJP_1M_######-'):]
        orig_nkjp_id = nkjp_orig_ids[nkjp_squeezed_ids.index(sqzd_nkjp_id)]
        skl_nkjp_sent_ids[orig_nkjp_id] = []
        nkjp_skl_ids[id] = orig_nkjp_id
    elif id in bonus_skladnica_mappings:
        orig_nkjp_id = bonus_skladnica_mappings[id]
        skl_nkjp_sent_ids[orig_nkjp_id] = []
        nkjp_skl_ids[id] = orig_nkjp_id
    else:
        print(id)

In [4]:
from lxml import etree
import os

In [5]:
# Get all the sents present in Składnica.
from operator import itemgetter

# NOTE this also contains empty lists where the sents were actually not annotated.
skl_sents = [] # pairs: word lemma, lexical unit id (or None)
skl_words = set() # all unique ones that are present
skl_synsets = set()

for skl_id in skl_ids:
    section_path = skladnica_path + skl_id
    # Each section has 1+ dirs of XML files.
    for dirname, _, __ in os.walk(section_path):
        if dirname != section_path: # ignore the dir itself
            # Get the XML files (each contains a sentence)
            for _, __, filenames in os.walk(dirname):
                for filename in filenames:
                    tree = etree.parse(dirname+'/'+filename)
                    # Check if the file has wordnet annotations.
                    if tree.find('.//plwn_interpretation') is None:
                        continue
                    
                    sent_id = filename[:-len('.xml')]
                    skl_nkjp_sent_ids[nkjp_skl_ids[skl_id]].append(sent_id)
                    
                    ###skl_nkjp_sent_ids[nkjp_skl_ids[skl_id]].append(tree.find(".//text"))
                    # Collect the list of sentence lemmas.
                    sent = []
                    for elem in tree.iterfind('node[@chosen]'):
                        if elem.attrib['chosen'] == 'true' and elem.find("terminal") is not None:
                            lemma = elem.find("terminal").find("base").text
                            lexical_id = None
                            if elem.find('.//luident') is not None:
                                lexical_id = elem.find('.//luident').text
                            sent.append((int(elem.attrib['from']), lemma, lexical_id))
                            skl_words.add(lemma)
                    sent.sort(key=itemgetter(0))
                    sent = [(token, lexical_id) for num, token, lexical_id in sent]
                    skl_sents.append(sent)

In [137]:
skl_sents[:2]

[[('bo', None),
  ('gdy', None),
  ('zostać', '74824'),
  ('proboszcz', '6516'),
  (',', None),
  ('nie', None),
  ('pozwolić', '6362'),
  ('rada', '7249'),
  ('parafialny', '376389'),
  ('wprowadzić', '80608'),
  ('się', None),
  ('procesja', '26994'),
  ('do', None),
  ('plebania', '28271'),
  ('i', None),
  ('kościół', '54419'),
  ('.', None)],
 [('-', None),
  ('taki', None),
  ('być', '81782'),
  ('u', None),
  ('my', '5224'),
  ('zwyczaj', '11870'),
  ('od', None),
  ('pokolenie', '20791'),
  ('.', None)]]

In [138]:
print(skl_nkjp_sent_ids)

{'GazetaPomorska': ['morph_6.39-s', 'morph_6.76-s', 'morph_6.22-s'], 'DP1980': ['morph_17.81-s', 'morph_17.75-s', 'morph_17.41-s', 'morph_3.55-s', 'morph_3.47-s', 'morph_7.68-s', 'morph_7.38-s', 'morph_7.47-s', 'morph_4.65-s', 'morph_4.19-s', 'morph_8.21-s', 'morph_8.63-s', 'morph_8.34-s'], '010-2-000000001': ['morph_3.20-s', 'morph_3.84-s'], 'ZycieWarszawy_Zycie': ['morph_146.82-s', 'morph_146.36-s', 'morph_146.28-s', 'morph_146.52-s', 'morph_146.41-s', 'morph_65.47-s', 'morph_65.15-s', 'morph_44.42-s', 'morph_44.47-s', 'morph_44.7-s', 'morph_44.14-s', 'morph_175.15-s', 'morph_85.9-s', 'morph_85.21-s', 'morph_85.28-s', 'morph_85.66-s', 'morph_135.81-s', 'morph_135.17-s', 'morph_135.8-s', 'morph_167.34-s', 'morph_167.52-s', 'morph_174.15-s', 'morph_174.45-s', 'morph_56.40-s', 'morph_56.29-s', 'morph_56.23-s', 'morph_56.17-s', 'morph_79.26-s', 'morph_79.23-s', 'morph_79.39-s', 'morph_17.57-s', 'morph_17.48-s', 'morph_3.77-s', 'morph_25.43-s', 'morph_25.11-s', 'morph_117.42-s', 'morph_11

In [63]:
print(skl_sents[:10])

[['bo', 'gdy', 'zostać', 'proboszcz', ',', 'nie', 'pozwolić', 'rada', 'parafialny', 'wprowadzić', 'się', 'procesja', 'do', 'plebania', 'i', 'kościół', '.'], ['-', 'taki', 'być', 'u', 'my', 'zwyczaj', 'od', 'pokolenie', '.'], [], ['po', 'awantura', 'z', 'rodzina', 'K', '.', 'wygrzebać', 'przeciwko', 'on', 'stary', 'sprawa', '.'], ['.'], ['osiągnąć', 'korzyść', 'prestiżowy', 'i', 'finansowy', '.'], [], ['twój', 'inicjatywa', ',', 'konkretny', ',', 'praktyczny', 'i', 'pożyteczny', ',', 'zyskać', 'uznanie', 'i', 'poparcie', '.'], [], []]


In [6]:
# Read the NKJP fragments, as lists of lemmas
unique_words = set()
words_count = 0
train_sents = []
rejected_sent_ids = []

with open(index_file) as index:
    for fragm_id in index:
        fragm_id = fragm_id.strip()
        # tag is namespaced, .// for finding anywhere in the tree
        tokens_filepath = nkjp_path+fragm_id+'/ann_morphosyntax.xml'
        if not os.path.isfile(tokens_filepath):
            print('Note: cannot access {}'.format(fragm_id.strip()))
            continue
        tree = etree.parse(tokens_filepath)
        for sent_subtree in tree.iterfind('.//{http://www.tei-c.org/ns/1.0}s[@{http://www.w3.org/XML/1998/namespace}id]'):
            
            # Skip sentences analyzed in Składnica.
            sent_id = sent_subtree.get('{http://www.w3.org/XML/1998/namespace}id').replace(' ', '_')
            if fragm_id in skl_nkjp_sent_ids and sent_id in skl_nkjp_sent_ids[fragm_id]:
                rejected_sent_ids.append(fragm_id+':'+sent_id)
                continue
                
            sent_lemmas = []
            for elem in sent_subtree.iterfind('.//{http://www.tei-c.org/ns/1.0}f[@name]'):
                if elem.attrib['name'] == 'base':
                    sent_lemmas.append(elem[0].text) # first child <string>
            train_sents += [[]]
            for word in sent_lemmas:
                train_sents[-1].append(word)
                words_count += 1
                unique_words.add(word)
print('{} individual sents rejected, for {} known from test corpus'.format(len(rejected_sent_ids), len(skl_sents)))

8211 individual sents rejected, for 8211 known from test corpus


In [80]:
words_count

1542059

## Get word vectors

In [7]:
import numpy as np

In [8]:
first_line = True
word_n = 0
word_to_idx = {}
idx_to_word = {}

# we'll read those from the data file
vecs_count = 0
vecs_dim = 100

# Get the vector word labels (we'll get vectors themselves in a momment).
with open(vecs_path+"data") as vecs_file:
    for line in vecs_file:
        if first_line:
            # Read metadata.
            vecs_count = int(line.split(' ')[0])
            vecs_dim = int(line.split(' ')[1])
            first_line = False
            continue
        # Read lemma base forms.
        word_to_idx[line.split(' ')[0]] = word_n
        idx_to_word[word_n] = line.split(' ')[0]
        word_n += 1

In [9]:
word_vecs = np.loadtxt(vecs_path+"data", encoding="utf-8",
                       dtype=np.float32, # tensorflow's requirement
                       skiprows=1, usecols=tuple(range(1, vecs_dim+1)))

In [10]:
# Add the dummy boundary/unknown marker.
word_vecs = np.vstack([word_vecs, np.zeros((1,vecs_dim), dtype=np.float32)])
vecs_count += 1

In [11]:
# Get the word's vector, or the dummy marker.
def word_id(word):
    return word_to_idx[word] if word in word_to_idx else vecs_count-1

In [12]:
vecs_count, vecs_dim

(1549323, 100)

## Keras - training the "language/gibberish discriminator"

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, Embedding
from keras.optimizers import SGD

Using TensorFlow backend.
/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [14]:
#
# Training corpus preparation.
#

from random import randint
from math import floor

# We need a special token for cases when the target word is near the start or end of sentence.
bound_token_id = vecs_count - 1 # the zero additional vector

sample_n = 0

train = np.zeros(((words_count + words_count // points_per_neg_sample) * corp_runs,
                  window_size * 2 + 1), dtype='int')
labels = np.ones(((words_count + words_count // points_per_neg_sample) * corp_runs,),
                dtype='int')

for run_n in range(corp_runs):
    sent_n = 0
    word_n = 0
        
    while sent_n < len(train_sents) and sample_n < train.shape[0]:
        # The positive sample.
        train[sample_n, window_size] = word_id(train_sents[sent_n][word_n]) # the target word
        for j in range(window_size): # its neighbors in the window_size
            train[sample_n, j] = (word_id(train_sents[sent_n][word_n-j-1]) if word_n-j-1 >= 0
                                  else bound_token_id)
            train[sample_n, window_size+j+1] = (word_id(train_sents[sent_n][word_n+j+1])
                                                if word_n+j+1 < len(train_sents[sent_n])
                                                else bound_token_id)
        # (the "one" label is the default)
        # labels[sample_n] = 1.0
        
        # The negative sample.
        sample_n += 1
        train[sample_n,] = np.random.permutation(train[sample_n-1,])
        # (replace two random words)
        train[sample_n, randint(0, window_size*2)] = randint(0, vecs_count-1)
        train[sample_n, randint(0, window_size*2)] = randint(0, vecs_count-1)
        labels[sample_n] = 0.0
                
        sample_n += 1
        word_n += 1
        # If words are exhausted, scan the sents for one that has some words.
        try:
            while word_n == len(train_sents[sent_n]):
                word_n = 0
                sent_n += 1
        # At the end of the corpus (== exhausted sents), break the loop.
        except IndexError:
            break

In [15]:
model = Sequential()                                                                                               
model.add(Embedding(vecs_count,
                    vecs_dim,
                    weights=[word_vecs],
                    input_length=window_size * 2 + 1,
                    trainable=False))                                                                              
model.add(LSTM(96))
model.add(Dense(1))
model.add(Activation('sigmoid'))

opt = SGD(lr=learning_rate, decay=reg_rate)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [16]:
model.fit(train, labels)

Epoch 1/1
7567012/7567012 [==============================] - 2061s 272us/step - loss: 0.1679 - acc: 0.9372


In [85]:
model

In [41]:
# cleanup to save some memory
train, labels = None, None

### (scribbling...)

In [27]:
X = np.asarray([word_id(w) for w in
                ['Niemcy', 'znienacka', 'wkroczyć', 'do', 'Francja', 'w', 'maj', 'kolejny', 'rok']],
               dtype='int')
X

array([  708, 21345,  5375,     8,   724,     0,   322,   164,    16])

In [36]:
Y = np.asarray([word_id(w) for w in
                ['kot', 'zimny', 'okrągły', 'start', 'do', 'w', 'Czechy', 'wykres', 'klub']],
               dtype='int')
Y
Y = np.asarray([word_id(w) for w in
                ['w', 'zimny', 'miesiąc', 'zakładać', 'żołnierz', 'do', 'szkoła', 'każdy', 'dzień']],
               dtype='int')
Y
Y = np.asarray([word_id(w) for w in
                ['w', 'kąt', 'izba', 'stać', 'urządzenie', 'kaflowy', 'z', 'wiele', 'garnek']],
               dtype='int')
Y

array([    0,  2605,   619,   123,  1300, 34251,     3,    92,  9152])

In [26]:
train.shape

(2646184, 9)

In [37]:
model.predict(np.atleast_2d(Y), batch_size=1)

array([[0.9920195]], dtype=float32)

## Loading Polish Wordnet 2 information

In [114]:
# Relations:
# only hyper/hyponymy, similarity, synonymy
rels1 = { '60', '11', '30' }
# should be rels1 + antonymy and conversion
rels2 = { '12', '101', '102', '104', '13' }
# should be rels1 + various meronymies and meronymy-like relations
rels3 = {'14', '15' # mero/holonymy
         '20', '21', '22', '23', '24', # meronymy types
         '64', '65', '80', # taxonomic type, causation
         '106', # type
         # verb meronymies
         '112', '113', '114', '115', '119', '120', '121', '122', '128', '129',
         '130', '131', '132', '134', '136', '137', '138', '140',
         # adjective relations
         '147', '150', '151', '152' }
# all relations
# rels4 is rels2 plus rels3

In [18]:
wordnet_xml = etree.parse(pl_wordnet)

In [19]:
from collections import defaultdict

In [22]:
def reverse_list_dict(dic):
    new_dic = defaultdict(list)
    for key, val in dic.items():
        for v in val:
            new_dic[v].append(key)
    return new_dic

In [115]:
# NOTE. Theoretically, we have lex_unit ids and synsets given by annotation, but we want
# to take into account all possibilities given by raw word forms. Our model will try to
# select the correct lex_unit ids.
# NOTE2. We store the bulk of relations in foo1 dicts, foo2 and foo3 store only ones specific to them

skl_word_wordids = defaultdict(list) # word -> ids as list
skl_wordid_synsets = defaultdict(list) # to harness synset relations

skl_neighbor_syns1 = defaultdict(list) # just for collecting them, synset ids
skl_neighbor_syns2 = defaultdict(list)
skl_neighbor_syns3 = defaultdict(list)
skl_wordid_neighbor_ids1 = defaultdict(list) # just for collecting them
skl_wordid_neighbor_ids2 = defaultdict(list)
skl_wordid_neighbor_ids3 = defaultdict(list)
skl_wordid_neighbors1 = defaultdict(list) # lemmas
skl_wordid_neighbors2 = defaultdict(list)
skl_wordid_neighbors3 = defaultdict(list)

# Wordnet lexical ID's of lemmas present in Składnica.
for lex_unit in wordnet_xml.iterfind('lexical-unit'):
    if lex_unit.get('name') in skl_words:
        skl_word_wordids[lex_unit.get('name')].append(lex_unit.get('id'))

In [116]:
# Compare words where we have their ids with the whole count
len(skl_word_wordids), len(skl_words)

(9954, 12895)

In [119]:
# words missing in wordnet
[w for w in skl_words if not w in skl_word_wordids]

['Chamula',
 'PG',
 'onejrologia',
 '1.500',
 'Byrcyn',
 'klima',
 'Kowal',
 'Małecki',
 None,
 'Domogarow',
 'Torba',
 'Wendy',
 '185-osobowy',
 'Jarzęcki',
 'Tatrzański',
 '271',
 'pewnie',
 'między',
 'Stanisław',
 'Haberda',
 'Rytwińska',
 'wezyrat',
 'pacyfistycznie',
 'miesięcznie',
 'Dominik',
 'Taribal',
 'Mińsk',
 'Młody',
 'Hoop',
 'połamać',
 'Kmetko',
 'Michalski',
 'solidnie',
 'przyśnić',
 '3364',
 'Daleki',
 'językowo',
 'rozmnoża',
 'Mróz',
 'tanio',
 '1979',
 'indziej',
 'rzeczywiście',
 'Tychowo',
 'niestety',
 'Nałkowska',
 'Baggio',
 'wymknąć',
 'skrupulatnie',
 'Raisa',
 'Kuzniecow',
 'typowo',
 'wywrzeć',
 'Faso',
 '1994',
 'BRH',
 'empatyczny',
 'aktywnie',
 'Steczkowska',
 'Kadyny',
 'malborski',
 'uśmiechnąć',
 'Kut',
 'Lubawski',
 'pokuracyjny',
 '1500',
 'Gustizia',
 'Piaseczno',
 'terrorystyczny',
 'Mirko',
 'Bogdan',
 'wszędzie',
 'AWF',
 'wspinać',
 'generalnie',
 'Kowalska',
 'włoszczowski',
 'Al-Qaida',
 'Hutnik',
 'MDM',
 'szokujący',
 'Risenius',
 'spo

In [120]:
# (reverse dict)
skl_wordid_words = reverse_list_dict(skl_word_wordids) # ids -> words

# Get the synsets.
for synset in wordnet_xml.iterfind('synset'):
    for unit in synset.iterfind('unit-id'):
        if unit.text in skl_wordid_words:
            skl_wordid_synsets[unit.text].append(synset.get('id')) # assign synset id to the wordid

In [121]:
# Distinct wordids (more than word forms) with known synsets
len(skl_wordid_synsets)

25766

In [122]:
# Get lexical relations.
for lexrel in wordnet_xml.iterfind('lexicalrelations'):
    if lexrel.get('parent') in skl_wordid_words:
        if lexrel.get('relation') in rels1:
            skl_wordid_neighbor_ids1[lexrel.get('parent')].append(lexrel.get('child'))
        if lexrel.get('relation') in rels2:
            skl_wordid_neighbor_ids2[lexrel.get('parent')].append(lexrel.get('child'))
        if lexrel.get('relation') in rels3:
            skl_wordid_neighbor_ids3[lexrel.get('parent')].append(lexrel.get('child'))
    if lexrel.get('child') in skl_wordid_words:
        if lexrel.get('relation') in rels1:
            skl_wordid_neighbor_ids1[lexrel.get('child')].append(lexrel.get('parent'))
        if lexrel.get('relation') in rels2:
            skl_wordid_neighbor_ids2[lexrel.get('child')].append(lexrel.get('parent'))
        if lexrel.get('relation') in rels3:
            skl_wordid_neighbor_ids3[lexrel.get('child')].append(lexrel.get('parent'))

In [123]:
# Wordids with purely lexical relations
len(skl_wordid_neighbor_ids1), len(skl_wordid_neighbor_ids2), len(skl_wordid_neighbor_ids3)

(2, 2756, 397)

In [124]:
# (reverse dict)
skl_synset_wordids = reverse_list_dict(skl_wordid_synsets)

In [125]:
# Get synset relations.
for synrel in wordnet_xml.iterfind('synsetrelations'):
    if synrel.get('parent') in skl_synset_wordids:
        if synrel.get('relation') in rels1:
            skl_neighbor_syns1[synrel.get('child')].append(synrel.get('parent'))
        if synrel.get('relation') in rels2:
            skl_neighbor_syns2[synrel.get('child')].append(synrel.get('parent'))
        if synrel.get('relation') in rels3:
            skl_neighbor_syns3[synrel.get('child')].append(synrel.get('parent'))
    if synrel.get('child') in skl_synset_wordids:
        if synrel.get('relation') in rels1:
            skl_neighbor_syns1[synrel.get('parent')].append(synrel.get('child'))
        if synrel.get('relation') in rels2:
            skl_neighbor_syns2[synrel.get('parent')].append(synrel.get('child'))
        if synrel.get('relation') in rels3:
            skl_neighbor_syns3[synrel.get('parent')].append(synrel.get('child'))

In [126]:
# Synsets with known relations
len(skl_neighbor_syns1), len(skl_neighbor_syns2), len(skl_neighbor_syns3)

(48222, 0, 7307)

In [127]:
# Get additional neighbor wordids from neighbor synsets.
for synset in wordnet_xml.iterfind('synset'):
    if synset.get('id') in skl_neighbor_syns1:
        for unit in synset.iterfind('unit-id'):
            for target_syns in skl_neighbor_syns1[synset.get('id')]:
                for receiver in skl_synset_wordids[target_syns]:
                    skl_wordid_neighbor_ids1[receiver].append(unit.text)
    if synset.get('id') in skl_neighbor_syns2:
        for unit in synset.iterfind('unit-id'):
            for target_syns in skl_neighbor_syns2[synset.get('id')]:
                for receiver in skl_synset_wordids[target_syns]:
                    skl_wordid_neighbor_ids2[receiver].append(unit.text)
    if synset.get('id') in skl_neighbor_syns1:
        for unit in synset.iterfind('unit-id'):
            for target_syns in skl_neighbor_syns3[synset.get('id')]:
                for receiver in skl_synset_wordids[target_syns]:
                    skl_wordid_neighbor_ids3[receiver].append(unit.text)

In [128]:
# How many wordids have neighbors in total
len(skl_wordid_neighbor_ids1), len(skl_wordid_neighbor_ids2), len(skl_wordid_neighbor_ids3)

(21886, 2756, 4494)

In [129]:
# (get reverses of these)
skl_neighbor_id_wordids1 = reverse_list_dict(skl_wordid_neighbor_ids1)
skl_neighbor_id_wordids2 = reverse_list_dict(skl_wordid_neighbor_ids2)
skl_neighbor_id_wordids3 = reverse_list_dict(skl_wordid_neighbor_ids3)

In [130]:
# How many *neighbors* we know
len(skl_neighbor_id_wordids1), len(skl_neighbor_id_wordids2), len(skl_neighbor_id_wordids3)

(70372, 3118, 7132)

In [131]:
# free some memory.
skl_neighbor_syns_wordids1 = None
skl_neighbor_syns_wordids2 = None
skl_neighbor_syns_wordids3 = None
skl_synset_wordids = None

In [132]:
# Finally, harness neighbors by their known wordids.
for lex_unit in wordnet_xml.iterfind('lexical-unit'):
    if lex_unit.get('id') in skl_neighbor_id_wordids1:
        for receiver in skl_neighbor_id_wordids1[lex_unit.get('id')]: # words "interested" in this neighbor
            skl_wordid_neighbors1[receiver].append(lex_unit.get('name'))
    if lex_unit.get('id') in skl_neighbor_id_wordids2:
        for receiver in skl_neighbor_id_wordids2[lex_unit.get('id')]:
            skl_wordid_neighbors2[receiver].append(lex_unit.get('name'))
    if lex_unit.get('id') in skl_neighbor_id_wordids3:
        for receiver in skl_neighbor_id_wordids3[lex_unit.get('id')]:
            skl_wordid_neighbors3[receiver].append(lex_unit.get('name'))

In [134]:
for word in skl_words:
    if word in skl_word_wordids:
        for (n, wid) in enumerate(skl_word_wordids[word]):
            print(word, wid, skl_wordid_neighbors1[wid])

łąka 11966 ['hala', 'hala pasterska', 'krajobraz', 'łąka łęgowa', 'łęg', 'okolica', 'pastwisko', 'polana', 'połonina', 'popas', 'strony', 'teren', 'turzycowisko']
waterman 272047 []
kolizja 2638 ['antynomia', 'antyteza', 'kontradyktoryjność', 'kontrast', 'niezgodność', 'przeciwieństwo', 'przeciwstawienie', 'przeciwstawność', 'rozbrat', 'sprzeczność']
kolizja 2639 ['incydent', 'wypadek']
butelka 19541 ['butelka', 'butelka do wina', 'butelka na wodę', 'butelka od piwa', 'butla', 'flaszka', 'litr', 'litrówka', 'małpka', 'manierka', 'naczynie', 'piersiówka', 'półlitrówka', 'syfon', 'szkło']
butelka 369059 ['igraszka', 'zabawa']
butelka 378150 ['flaszka', 'flaszka', 'napój', 'porcja napoju']
butelka 378873 ['butelka', 'butelka antykolkowa']
kłócić 76292 ['antagonizować', 'CZASOWNIK - DZIAŁANIE NDK oznaczający powodowanie zmiany sytuacji związanych ze współżyciem w społeczeństwie', 'dzielić', 'rozdzielać', 'wadzić', 'wadzić', 'waśnić', 'waśnić']
kłócić 94116 ['burzyć', 'łamać', 'mieszać', 'n

kąpiel 103367 ['kąpiel barwiąca', 'kąpiel farbiarska', 'kąpiel fotograficzna', 'kąpiel ołowiana', 'kąpiel solna', 'roztwór']
triumfować 75622 ['wygrywać']
triumfować 75626 ['chełpić się', 'chlubić się', 'chwalić się', 'przechwalać się', 'reklamować się', 'szczycić się']
triumfować 75634 ['brać górę', 'przeważać', 'zwyciężać']
decyzja 1024 ['dokument', 'papier', 'werdykt']
decyzja 17812 ['ewaluacja', 'niezgoda', 'ocena', 'orzeczenie', 'postanowienie', 'pozwolenie', 'przyzwolenie', 'rozsądzenie', 'rozstrzygnięcie', 'wybór', 'zezwolenie', 'zgoda']
chodzić 724 ['sprzedawać się']
chodzić 725 ['CZASOWNIK - ZDARZENIE NDK oznaczający zjawiska fizyczne', 'kontaktować', 'pracować', 'prowadzić się']
chodzić 19566 ['brodzić', 'człapać', 'dreptać', 'drobić', 'kusztykać', 'kuśtykać', 'maszerować', 'poruszać się', 'poruszać się', 'przemieszczać się', 'przenosić się', 'stąpać', 'tupotać', 'tuptać']
chodzić 77935 ['uczęszczać']
chodzić 77936 ['bywać']
chodzić 77937 ['przemieszczać się']
chodzić 77938 [

samozaparcie 49430 ['benedyktyńska cierpliwość', 'cecha czynności', 'cecha działania', 'cierpliwość', 'hart', 'niewzruszoność', 'odporność', 'ortodoksyjność', 'pracowitość', 'systematyczność', 'twardość', 'usilność', 'usilność', 'wola', 'wytrzymałość']
ośmieszyć 86880 ['obśmiać', 'wyśmiać', 'zadrwić']
argumentować 155 ['dowodzić', 'klarować', 'legitymować', 'obalać', 'objaśniać', 'popierać', 'racjonalizować', 'tłumaczyć', 'tłumaczyć', 'udowadniać', 'usprawiedliwiać', 'wyjaśniać', 'wykazywać', 'wywodzić', 'zbijać']
mozół 3627 ['fatyga', 'nakład sił', 'trud', 'wysiłek']
życiowy 11883 ['biopsychiczny']
życiowy 23114 ['dorzeczny', 'logiczny', 'pragmatyczny', 'sensowny', 'zdroworozsądkowy']
życiowy 23113 ['praktyczny', 'przedsiębiorczy', 'rzutki', 'zorganizowany']
życiowy 22186 []
targ 8920 ['giełda', 'obiekt handlowy', 'pchli targ', 'plac', 'suk', 'targowica', 'targowica', 'zielony rynek']
targ 8921 ['jarmark', 'kiermasz', 'sprzedanie', 'sprzedaż']
upadek 9420 ['amoralność', 'dehumanizacja

dostawa 28352 ['import', 'przewoźnictwo', 'przewóz', 'transport']
dostawa 65936 ['cos', 'cos', 'cosinus', 'cosinus', 'kosinus', 'kosinus']
zastać 74601 ['przyłapać', 'zdybać', 'znaleźć']
władza 10732 ['aparat', 'biurokracja', 'centrala', 'dyrektorium', 'dyrektorium', 'grupa', 'grupa ludzi', 'hierarchia', 'ludzie', 'oligarchia', 'rządzący', 'technokracja', 'teren', 'zarząd']
władza 10733 ['absolutyzm', 'autarchia', 'autokracja', 'autokratyzm', 'dostojeństwo', 'dowództwo', 'dwuwładza', 'dyktatura proletariatu', 'godność', 'jednowładztwo', 'jedynowładztwo', 'kadencja', 'kierownictwo', 'koregencja', 'legitymacja', 'monarchia absolutna', 'okupacja', 'pajdokracja', 'prawo', 'protektorat', 'pryncypat', 'rząd', 'rząd', 'rząd dusz', 'samodzierżawie', 'samodzierżawienie', 'samowładztwo', 'uprawnienie', 'urząd', 'władza absolutna', 'władza ludowa', 'współrząd', 'współwładztwo']
władza 27759 ['jurysdykcja', 'legislatura', 'sądownictwo', 'struktura', 'system', 'układ', 'władza ustawodawcza']
władza

amarantowy 63892 ['różowoczerwony']
perfumy 5482 ['kosmetyk zapachowy', 'zapach']
prywatyzacja 29824 ['przekształcenie własnościowe', 'reprywatyzacja']
morze 12588 ['akwen', 'morze epikontynentalne', 'zbiornik wodny']
morze 12854 ['bezlik', 'bezmiar', 'bezmierność', 'fura', 'góra', 'krocie', 'kupa', 'masa', 'mnóstwo', 'nieskończoność', 'ogrom', 'piramida', 'sterta', 'stos', 'wielość', 'zatrzęsienie']
suchy 22445 ['spopielały', 'wyschnięty']
suchy 23083 ['nieświeży', 'stary']
suchy 23082 []
suchy 23081 ['mieszany', 'sypki', 'zeschły', 'zeschnięty']
suchy 375965 ['pustynny']
suchy 375966 []
suchy 375967 ['cichy', 'niesłyszalny', 'niewyraźny', 'płynny']
suchy 375968 ['szczupły']
suchy 375969 ['chudy']
suchy 375970 []
poświęcać 6148 ['ofiarowywać', 'wyrzekać się']
poświęcać 75992 ['dedykować', 'koncentrować', 'przeznaczać', 'skupiać']
poświęcać 82156 ['konsekrować', 'wykonywać czynności religijne bądź magiczne']
umilknąć 71860 ['nabrać wody w usta', 'przestać', 'przymknąć się', 'przymknąć 

...

wprowadzić 82878 ['przyłączyć', 'wcielić', 'wgrać', 'włączyć', 'zaksięgować', 'zarejestrować']
wprowadzić 82879 ['zakłócić']
powszedni 20872 ['banalny', 'codzienny', 'niepozorny', 'niewyrafinowany', 'prozaiczny', 'przyziemny', 'standardowy', 'statystyczny', 'typowy']
niecodzienny 359990 ['nietypowy', 'specyficzny', 'swoisty']
październikowy 145334 ['jesienny']
zaklinować 97932 ['przymocować', 'przytwierdzić', 'umocować', 'zamocować']
zaklinować 97935 ['zablokować', 'zagrodzić', 'zatamować', 'zatarasować']
wnętrze 9956 ['miejsce', 'ośródka', 'rdzeń']
wnętrze 65885 ['pomieszczenie mieszkalne']
utrudniać 9583 ['CZASOWNIK - CZYNNOŚĆ NDK oznaczający powodowanie zm. sytuacji związanej z łańcuchem przyczynowo-skutkowym lub czasowym', 'głuszyć', 'gmatwać', 'hermetyzować', 'komplikować', 'plątać', 'przesłaniać', 'przeszkadzać', 'przysłaniać', 'rzucać kłody pod nogi', 'sabotować', 'torpedować', 'wikłać', 'zagłuszać', 'zapętlać', 'zawadzać']
slalomista 56666 ['sportowiec', 'sportsmen']
pozostawać

zwariować 71587 ['dostać małpiego rozumu', 'stracić głowę', 'stracić panowanie nad sobą']
wirus 9936 ['adenowirus', 'bakteriofag', 'drobnoustrój', 'ebola', 'enterowirus', 'fag', 'HIV', 'retrowirus', 'wirion']
Eiffel 283566 []
wyzwolić 10656 ['nakręcić', 'naładować', 'napełnić', 'nastroić', 'nasunąć', 'podrażnić', 'poruszyć', 'przejąć', 'przywiązać', 'rozpalić', 'rozpłomienić', 'ruszyć', 'uczulić', 'uwolnić', 'uwrażliwić', 'wlać', 'wstrząsnąć', 'wyczulić', 'wywołać', 'wzbudzić uczucia negatywne', 'wzbudzić uczucia pozytywne', 'zachęcić się', 'zaciekawić', 'zadziwić', 'zafrapować', 'zainteresować', 'zająć', 'zaskoczyć', 'zdumieć', 'zdziwić']
wyzwolić 87278 ['dopomóc', 'odbić', 'pomóc', 'poratować', 'przyjść z pomocą', 'udzielić pomocy', 'wesprzeć', 'wspomóc', 'wykupić']
wyzwolić 87298 ['CZASOWNIK - DZIAŁANIE DK oznaczający spowodowanie zmiany położenia lub relacji przestrzennych', 'odblokować', 'rozkuć', 'rozpętać', 'rozprząc', 'spuścić']
wywalić 18666 ['pozbyć się', 'rozstać się', 'wyzb

starowina 64849 []
podroby 23763 ['mięso']
przezwyciężyć 83135 ['dać sobie radę', 'obronić się', 'oprzeć się', 'poradzić sobie', 'przełamać się', 'przemóc się', 'przetrzymać', 'wymanewrować', 'wziąć', 'zwalczyć', 'zwalczyć']
macho 160806 []
macho 276467 []
przywalić 71441 ['dać', 'dać łupnia', 'pieprznąć', 'pieprznąć', 'pierdolnąć', 'pierdolnąć', 'przypieprzyć', 'przypieprzyć', 'przypierdolić', 'przypierdolić', 'przypierdzielić', 'przypierdzielić', 'przypierniczyć', 'przypierniczyć', 'uderzyć', 'uderzyć', 'wymierzyć', 'zadać cios', 'zadać raz']
przywalić 76284 ['dać do wiwatu', 'dać popalić', 'dać się we znaki', 'dojeść', 'dokuczyć', 'dopiec', 'przygnębić', 'zgnębić']
przywalić 86526 ['przygnieść']
przywalić 86527 ['dorzucić', 'podrzucić']
przywalić 86528 ['obciążyć']
nadleśnictwo 28618 ['urząd']
nadleśnictwo 28619 ['urząd']
nadleśnictwo 28620 ['jednostka administracyjna']
telekomunikacja 8949 ['komunikacja', 'komunikowanie się', 'kontaktowanie się', 'porozumiewanie się', 'radiokomunik

jednostka 2366 ['albinos', 'anaerobiont', 'asymilator', 'autotrof', 'beztlenowiec', 'bioindykator', 'czynnik biotyczny', 'czynnik biotyczny', 'destruent', 'diploid', 'egzotyk', 'ekotyp', 'embrion', 'endemit', 'entomofag', 'gatunek endemiczny', 'gospodarz', 'hermafrodyta', 'heterotrof', 'heterozygota', 'higrofil', 'homozygota', 'istota żywa', 'jednokomórkowiec', 'klon', 'koński organizm', 'kosmopolita', 'kserofob', 'kseroterm', 'mikroorganizm', 'mutant', 'nosiciel', 'obiekt organiczny', 'obojnak', 'organizm diploidalny', 'organizm jednokomórkowy', 'organizm kosmopolityczny', 'organizm kserotermiczny', 'organizm wielokomórkowy', 'organizm wskaźnikowy', 'plechowiec', 'praorganizm', 'preparat', 'reducent', 'regenerator', 'roczniak', 'roślina', 'roztocz', 'roztocze', 'saprofit', 'stworzenie', 'stworzenie oceniane negatywnie', 'symbiont', 'tetraploid', 'tkankowiec', 'twór', 'typ ekologiczny', 'wielokomórkowiec', 'zarodek', 'zooid', 'żywiciel']
jednostka 2367 ['całość', 'jednostka administrac

start 345331 []
start 345778 []
start 346719 []
start 347430 []
start 349941 []
start 352028 []
start 352069 []
start 352071 []
chemioterapia 46855 ['kuracja', 'terapia']
alokacja 39781 ['fragmentacja', 'kolportaż', 'kolporterstwo', 'podział', 'proliferacja', 'redystrybucja', 'rozdawnictwo', 'rozdzielnictwo', 'segmentacja']
rojenie 55632 ['proces biologiczny', 'uniezależnienie', 'usamodzielnienie']
rojenie 113676 ['dumanie', 'medytowanie', 'mniemanie', 'myślenie', 'rozmyślanie', 'rozumowanie']
ciąża 791 ['ciąża mnoga', 'ciąża pozamaciczna', 'ciąża przenoszona', 'ciąża wieloraka', 'proces fizjologiczny', 'wpadka']
ciąża 67993 ['czas', 'moment', 'okres', 'period', 'pora']
absolwentka 25135 ['ośmioklasistka']
cichy 796 ['bezdźwięczny', 'bezgłośny', 'bezszelestny', 'głuchy', 'matowy', 'przyciszony', 'przytłumiony', 'stłumiony', 'suchy']
cichy 18963 ['nieoficjalny', 'podskórny', 'zakryty', 'zakryty']
cichy 18965 ['bukoliczny', 'idylliczny', 'miły', 'odludny', 'przyjazny', 'przyjemny', 'siel